In [87]:
import json
from itertools import product

f = open('../../app/data/feature_classes.json')
raw = f.read()

In [88]:
objs = json.loads(raw)

In [89]:
# TODO a lot of these should not be among our autosuggest
# Might also want to compute the likelihood that any of these are actually in the area
words = [x for x in objs.keys()]
ops = ['in [[zip code]]', 'in [[county]]', 'in [[area code]]', 'in [[neighborhood]]', 'in [[city]]']
autocomplete_suggestions = [' '.join(x) for x in product(words, ops)]
np.random.choice(autocomplete_suggestions, 10)

array(['nurserys in [[neighborhood]]', 'sports pitch in [[city]]',
       'conference center in [[city]]', 'arts centres in [[county]]',
       'level crossings in [[city]]', 'coastlines in [[neighborhood]]',
       'department store in [[county]]', 'cash in [[zip code]]',
       'key cutter in [[area code]]', 'kiosk shop in [[county]]'],
      dtype='<U47')

In [90]:
autocomplete_suggestions = [' '.join(x) for x in product(words, ['within [[x]] meters of', 'near to', 'within an [[x]] minute drive of'], words)]
np.random.choice(autocomplete_suggestions, 10)

array(['plaques near to rubbish bin',
       'brewery within [[x]] meters of land',
       'buildings within an [[x]] minute drive of drains',
       'community center within an [[x]] minute drive of platform',
       'conservations near to atm',
       'conference centers within [[x]] meters of guided bus lanes',
       'animal shelter near to zip lines',
       'windmill within an [[x]] minute drive of hamlets',
       'brewery near to restaurant',
       'fashion shop within [[x]] meters of cosmetics shops'],
      dtype='<U87')

In [92]:
import shapely

In [100]:
import geopandas as gpd
gpd.datasets.available

['naturalearth_cities', 'naturalearth_lowres', 'nybb']

In [101]:
# gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

,name,geometry
0,Vatican City,POINT (12.45339 41.90328)
1,San Marino,POINT (12.44177 43.93610)
2,Vaduz,POINT (9.51667 47.13372)
3,Lobamba,POINT (31.20000 -26.46667)
4,Luxembourg,POINT (6.13000 49.61166)
...,...,...
238,Rio de Janeiro,POINT (-43.21212 -22.90731)
239,São Paulo,POINT (-46.62697 -23.55673)
240,Sydney,POINT (151.21255 -33.87137)
241,Singapore,POINT (103.85387 1.29498)


In [113]:
from sqlalchemy import create_engine
# Generate neighborhoods

sql = '''
  WITH centroids AS (
    SELECT ST_Collect(ST_Centroid(geom)) AS collect_geom
    FROM osm
    WHERE 1=1
      AND tags::varchar ILIKE '%San%Francisco%' AND tags::varchar ILIKE '%neighbourhood%'
  )
  SELECT ST_VoronoiPolygons()
  FROM centroids
'''

conn = create_engine('postgresql://osmuser:@localhost:5432/osm')
gpd.read_postgis(sql, conn)

ModuleNotFoundError: No module named 'psycopg2'

In [117]:
a = {"type":"GeometryCollection","geometries":[{"type":"Polygon","coordinates":[[[-122.6050404,37.9127752],[-122.496375373,37.9127752],[-122.473368759,37.834017003],[-122.473834399,37.795329998],[-122.487676794,37.770230205],[-122.6050404,37.798174097],[-122.6050404,37.9127752]]]},{"type":"Polygon","coordinates":[[[-122.6050404,37.798174097],[-122.487676794,37.770230205],[-122.477282469,37.761541957],[-122.480267656,37.746073069],[-122.512970069,37.698941766],[-122.6050404,37.611774008],[-122.6050404,37.798174097]]]},{"type":"Polygon","coordinates":[[[-122.6050404,37.5995868],[-122.6050404,37.611774008],[-122.512970069,37.698941766],[-122.46505625,37.722997003],[-122.447349297,37.714545991],[-122.473348075,37.5995868],[-122.6050404,37.5995868]]]},{"type":"Polygon","coordinates":[[[-122.477282469,37.761541957],[-122.487676794,37.770230205],[-122.473834399,37.795329998],[-122.463999882,37.78018916],[-122.461651816,37.770841121],[-122.462311848,37.769248468],[-122.47710748,37.761589449],[-122.477282469,37.761541957]]]},{"type":"Polygon","coordinates":[[[-122.473834399,37.795329998],[-122.473368759,37.834017003],[-122.450023519,37.796921635],[-122.463999882,37.78018916],[-122.473834399,37.795329998]]]},{"type":"Polygon","coordinates":[[[-122.47710748,37.761589449],[-122.462311848,37.769248468],[-122.446958366,37.757590229],[-122.444160383,37.747395286],[-122.448437631,37.742158585],[-122.47710748,37.761589449]]]},{"type":"Polygon","coordinates":[[[-122.449946344,37.738818357],[-122.454169638,37.736994636],[-122.480267656,37.746073069],[-122.477282469,37.761541957],[-122.47710748,37.761589449],[-122.448437631,37.742158585],[-122.449946344,37.738818357]]]},{"type":"Polygon","coordinates":[[[-122.454169638,37.736994636],[-122.449946344,37.738818357],[-122.443778894,37.728684207],[-122.444532639,37.717534615],[-122.447349297,37.714545991],[-122.46505625,37.722997003],[-122.454169638,37.736994636]]]},{"type":"Polygon","coordinates":[[[-122.512970069,37.698941766],[-122.480267656,37.746073069],[-122.454169638,37.736994636],[-122.46505625,37.722997003],[-122.512970069,37.698941766]]]},{"type":"Polygon","coordinates":[[[-122.416201435,37.9127752],[-122.423356267,37.800199886],[-122.448824394,37.795667953],[-122.450023519,37.796921635],[-122.473368759,37.834017003],[-122.496375373,37.9127752],[-122.416201435,37.9127752]]]},{"type":"Polygon","coordinates":[[[-122.462311848,37.769248468],[-122.461651816,37.770841121],[-122.451332945,37.77765491],[-122.439151623,37.773334915],[-122.438920439,37.772082787],[-122.439962743,37.766761978],[-122.446958366,37.757590229],[-122.462311848,37.769248468]]]},{"type":"Polygon","coordinates":[[[-122.449946344,37.738818357],[-122.448437631,37.742158585],[-122.444160383,37.747395286],[-122.42779064,37.747695515],[-122.427649084,37.747610898],[-122.420813897,37.72700708],[-122.443778894,37.728684207],[-122.449946344,37.738818357]]]},{"type":"Polygon","coordinates":[[[-122.461651816,37.770841121],[-122.463999882,37.78018916],[-122.450023519,37.796921635],[-122.448824394,37.795667953],[-122.447334878,37.791790871],[-122.451332945,37.77765491],[-122.461651816,37.770841121]]]},{"type":"Polygon","coordinates":[[[-122.473348075,37.5995868],[-122.447349297,37.714545991],[-122.444532639,37.717534615],[-122.424298259,37.712311183],[-122.413545834,37.5995868],[-122.473348075,37.5995868]]]},{"type":"Polygon","coordinates":[[[-122.451332945,37.77765491],[-122.447334878,37.791790871],[-122.437270239,37.785433579],[-122.43521317,37.779647325],[-122.439151623,37.773334915],[-122.451332945,37.77765491]]]},{"type":"Polygon","coordinates":[[[-122.444532639,37.717534615],[-122.443778894,37.728684207],[-122.420813897,37.72700708],[-122.419366713,37.725412107],[-122.424298259,37.712311183],[-122.444532639,37.717534615]]]},{"type":"Polygon","coordinates":[[[-122.437270239,37.785433579],[-122.447334878,37.791790871],[-122.448824394,37.795667953],[-122.423356267,37.800199886],[-122.422501484,37.797507288],[-122.437270239,37.785433579]]]},{"type":"Polygon","coordinates":[[[-122.446958366,37.757590229],[-122.439962743,37.766761978],[-122.42700377,37.761136611],[-122.42779064,37.747695515],[-122.444160383,37.747395286],[-122.446958366,37.757590229]]]},{"type":"Polygon","coordinates":[[[-122.421827305,37.785791887],[-122.422645765,37.784012048],[-122.428850585,37.779217085],[-122.43521317,37.779647325],[-122.437270239,37.785433579],[-122.422501484,37.797507288],[-122.421009717,37.795517104],[-122.421827305,37.785791887]]]},{"type":"Polygon","coordinates":[[[-122.422645765,37.784012048],[-122.421827305,37.785791887],[-122.408679295,37.777882576],[-122.407959821,37.769105506],[-122.408465511,37.768629594],[-122.415774135,37.769500731],[-122.422645765,37.784012048]]]},{"type":"Polygon","coordinates":[[[-122.419366713,37.725412107],[-122.420813897,37.72700708],[-122.427649084,37.747610898],[-122.408677304,37.752545974],[-122.400421048,37.745380359],[-122.405814546,37.727524955],[-122.419366713,37.725412107]]]},{"type":"Polygon","coordinates":[[[-122.439962743,37.766761978],[-122.438920439,37.772082787],[-122.423551465,37.767987087],[-122.423343571,37.767784706],[-122.42700377,37.761136611],[-122.439962743,37.766761978]]]},{"type":"Polygon","coordinates":[[[-122.42700377,37.761136611],[-122.423343571,37.767784706],[-122.415774135,37.769500731],[-122.408465511,37.768629594],[-122.408677304,37.752545974],[-122.427649084,37.747610898],[-122.42779064,37.747695515],[-122.42700377,37.761136611]]]},{"type":"Polygon","coordinates":[[[-122.438920439,37.772082787],[-122.439151623,37.773334915],[-122.43521317,37.779647325],[-122.428850585,37.779217085],[-122.423551465,37.767987087],[-122.438920439,37.772082787]]]},{"type":"Polygon","coordinates":[[[-122.423343571,37.767784706],[-122.423551465,37.767987087],[-122.428850585,37.779217085],[-122.422645765,37.784012048],[-122.415774135,37.769500731],[-122.423343571,37.767784706]]]},{"type":"Polygon","coordinates":[[[-122.421827305,37.785791887],[-122.421009717,37.795517104],[-122.405330386,37.790893417],[-122.408679295,37.777882576],[-122.421827305,37.785791887]]]},{"type":"Polygon","coordinates":[[[-122.292834354,37.9127752],[-122.402022098,37.792206512],[-122.405330386,37.790893417],[-122.421009717,37.795517104],[-122.422501484,37.797507288],[-122.423356267,37.800199886],[-122.416201435,37.9127752],[-122.292834354,37.9127752]]]},{"type":"Polygon","coordinates":[[[-122.407959821,37.769105506],[-122.408679295,37.777882576],[-122.405330386,37.790893417],[-122.402022098,37.792206512],[-122.393484601,37.778324245],[-122.402201059,37.769898699],[-122.407959821,37.769105506]]]},{"type":"Polygon","coordinates":[[[-122.408465511,37.768629594],[-122.407959821,37.769105506],[-122.402201059,37.769898699],[-122.394180372,37.76495527],[-122.39211747,37.74754657],[-122.400421048,37.745380359],[-122.408677304,37.752545974],[-122.408465511,37.768629594]]]},{"type":"Polygon","coordinates":[[[-122.2756677,37.5995868],[-122.413545834,37.5995868],[-122.424298259,37.712311183],[-122.419366713,37.725412107],[-122.405814546,37.727524955],[-122.2756677,37.618781193],[-122.2756677,37.5995868]]]},{"type":"Polygon","coordinates":[[[-122.2756677,37.9127752],[-122.2756677,37.805676833],[-122.364605026,37.78064734],[-122.393484601,37.778324245],[-122.402022098,37.792206512],[-122.292834354,37.9127752],[-122.2756677,37.9127752]]]},{"type":"Polygon","coordinates":[[[-122.402201059,37.769898699],[-122.393484601,37.778324245],[-122.364605026,37.78064734],[-122.388344932,37.766095892],[-122.394180372,37.76495527],[-122.402201059,37.769898699]]]},{"type":"Polygon","coordinates":[[[-122.2756677,37.618781193],[-122.405814546,37.727524955],[-122.400421048,37.745380359],[-122.39211747,37.74754657],[-122.384469492,37.746926113],[-122.2756677,37.723457188],[-122.2756677,37.618781193]]]},{"type":"Polygon","coordinates":[[[-122.2756677,37.723457188],[-122.384469492,37.746926113],[-122.388344932,37.766095892],[-122.364605026,37.78064734],[-122.2756677,37.805676833],[-122.2756677,37.723457188]]]},{"type":"Polygon","coordinates":[[[-122.39211747,37.74754657],[-122.394180372,37.76495527],[-122.388344932,37.766095892],[-122.384469492,37.746926113],[-122.39211747,37.74754657]]]}]}

In [130]:
gpd.GeoDataFrame({'type': "Feature", 'features': [x for x in a['geometries']]}).plot()

ImportError: The matplotlib package is required for plotting in geopandas. You can install it using 'conda install -c conda-forge matplotlib' or 'pip install matplotlib'.

In [131]:
pip install matplotlib

ERROR: Could not find a version that satisfies the requirement matplotlib (from versions: none)
ERROR: No matching distribution found for matplotlib
Note: you may need to restart the kernel to use updated packages.
